In [ ]:
import os
import mujoco
import numpy as np
from gymnasium.envs.mujoco import MujocoEnv
from gymnasium import spaces
from gymnasium.spaces import Box
from typing import Dict, Union


DEFAULT_CAMERA_CONFIG = {
    "trackbodyid": -1,
    "distance": 4.0,
}

class MyRobotEnv(MujocoEnv):
    def __init__(
        self, 
        xml_path="../assets/crane_x7.urdf.xml",
        frame_skip: int = 5,
        default_camera_config: Dict[str, Union[float, int]] = DEFAULT_CAMERA_CONFIG,
        **kwargs,
    ):
        # 一度モデルを読み込んで観測次元を取得
        model = mujoco.MjModel.from_xml_path(xml_path)
        obs_dim = model.nq + model.nv
        observation_space = Box(low=-np.inf, high=np.inf, shape=(obs_dim,), dtype=np.float32)

        # 行動空間：joint velocities [-1, 1] 正規化
        # 現状はmodelのアクチュエータが認識していないので、model.nuがなくなっている
        # そのため, model.ctrlもないのでstepができない状態
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(model.nu,), dtype=np.float32
        )
        print(self.action_space)
        
        # 親クラスの初期化
        super().__init__(
            model_path=xml_path,
            frame_skip=frame_skip,
            observation_space=observation_space,
            render_mode="human",
            default_camera_config=default_camera_config,
            **kwargs,
        )
        self.metadata = {
            "render_modes": [
                "human",
                "rgb_array",
                "depth_array",
                "rgbd_tuple",
            ],
            "render_fps": int(np.round(1.0 / self.dt)),
        }

    def step(self, action):
        # 正規化されたactionをスケーリング
        scaled_action = action * self.model.actuator_ctrlrange[:, 1]
        self.do_simulation(scaled_action, self.frame_skip)

        obs = self._get_obs()
        reward = self._compute_reward(obs, action)
        done = False
        info = {}

        return obs, reward, done, info

    def _get_obs(self):
        return np.concatenate([self.data.qpos.flat, self.data.qvel.flat])

    def _compute_reward(self, obs, action):
        # エンドエフェクタの位置を使用した報酬例
        end_effector_pos = self.sim.data.site_xpos[self.model.site_name2id("ee_site")]
        goal = np.array([0.5, 0.0, 0.2])
        dist = np.linalg.norm(end_effector_pos - goal)
        return -dist

    def reset_model(self):
        # ランダム初期化
        qpos = self.init_qpos + np.random.uniform(-0.01, 0.01, size=self.model.nq)
        qvel = self.init_qvel + np.random.uniform(-0.01, 0.01, size=self.model.nv)
        self.set_state(qpos, qvel)
        return self._get_obs()

if __name__=='__main__':
    env = MyRobotEnv()
    env.reset()
    env.render()

: 